In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [2]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [3]:
tf.keras.backend.set_floatx('float64')

In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(3,)),
        layers.Dense(2, name="layer1"),
    ]
)

In [5]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2, n_external=1)

In [7]:
n_ode.pretrain()

tf.Tensor([-0.13708417  0.22908976], shape=(2,), dtype=float64)
tf.Tensor([ 0.01636936 -0.06492924], shape=(2,), dtype=float64)
tf.Tensor([ 0.06829486 -0.16639969], shape=(2,), dtype=float64)
tf.Tensor([ 0.06684586 -0.16232818], shape=(2,), dtype=float64)
tf.Tensor([ 0.04822951 -0.12286619], shape=(2,), dtype=float64)
tf.Tensor([ 0.02774415 -0.07850376], shape=(2,), dtype=float64)
tf.Tensor([ 0.01012668 -0.03924915], shape=(2,), dtype=float64)
tf.Tensor([-0.00372692 -0.00739726], shape=(2,), dtype=float64)
tf.Tensor([-0.01404717  0.01716939], shape=(2,), dtype=float64)
tf.Tensor([-0.02131578  0.03519688], shape=(2,), dtype=float64)


In [37]:
# Generate solution

In [38]:
#
N_n = int(2)
c = 0.4
k = 4.0
def oscilator(t, y, f):
    f_val = f(t)
    return np.array([y[1], -c*y[1]-k*y[0]+f_val])
t_final = 200.0
n_eval = int(1000)
t_span = np.array([0.0, t_final])
t_eval = np.linspace(0, t_final, num=n_eval)
y0 = np.array([1.0, 0.0])
#
f_ext = np.vstack(( np.zeros((200,1)), np.ones((400,1))*0.5, np.ones((400,1)) ) )
f_interp = interp1d(t_eval, f_ext, kind='linear', axis=0)

In [39]:
func_1 = lambda t,y: oscilator(t,y,f_interp)
sol = solve_ivp(func_1, t_span, y0, t_eval=t_eval)

<ipython-input-38-7ae9c9a7a2c5>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([y[1], -c*y[1]-k*y[0]+f_val])


In [40]:
plt.plot(sol.t, sol.y[0,:])

In [41]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
#
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )
f_ext_tf = tf.constant(f_ext)

In [42]:
# interpolation in tf
f_ext_interp_np = interp1d(t_eval, f_ext, kind='linear', axis=0)
f_ext_interp = lambda t: tf.expand_dims(tf.constant(f_ext_interp_np(t)), axis=0)

In [43]:
sol1 = n_ode.forward_solve(t_target, y_target[0,:], x_external=f_ext_interp)
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [ ]:
# pretrain

In [ ]:
# adjoint method

In [ ]:
loss, dl_dy, a = n_ode.adjoint_method(t_target[0:3], y_target[0:3,:])

In [ ]:
# fit

In [44]:
n_epoch = 40
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=5, adjoint_method=False, x_external=f_ext_tf)

--- Epoch #1 ---
Total loss of epoch: 0.27813935256563127
Elapsed time for epoch: 6.419644832611084
--- Epoch #2 ---
Total loss of epoch: 0.1632336052134633
Elapsed time for epoch: 5.068451881408691
--- Epoch #3 ---
Total loss of epoch: 0.11598923848941922
Elapsed time for epoch: 5.496361017227173
--- Epoch #4 ---
Total loss of epoch: 0.10611814400181174
Elapsed time for epoch: 5.884210824966431
--- Epoch #5 ---
Total loss of epoch: 0.09845541452523321
Elapsed time for epoch: 6.177792072296143
--- Epoch #6 ---
Total loss of epoch: 0.09846931812353432
Elapsed time for epoch: 5.3530638217926025
--- Epoch #7 ---
Total loss of epoch: 0.0887346864619758
Elapsed time for epoch: 5.9479429721832275
--- Epoch #8 ---
Total loss of epoch: 0.08960514192585833
Elapsed time for epoch: 5.546143531799316
--- Epoch #9 ---
Total loss of epoch: 0.08926902036182582
Elapsed time for epoch: 5.305544137954712
--- Epoch #10 ---
Total loss of epoch: 0.08803501449438045
Elapsed time for epoch: 6.836256980895996

In [ ]:
# Check derivatives

In [45]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float64, numpy=array([0., 0.])>

In [46]:
loss, dl_dp = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)
#loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target, x_external=f_ext_interp)
#dl_dp = a[2:]

In [47]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c+dp], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float64, numpy=array([0., 0.])>

In [48]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)

In [49]:
(loss2-loss)/dp

<tf.Tensor: shape=(), dtype=float64, numpy=0.008684583008289337>

In [50]:
dl_dp

[<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[ 4.37851349e-03, -1.01596591e-02],
        [ 3.64566900e-02,  8.68960490e-03],
        [ 5.37464029e-04,  3.81110846e-05]])>,
 <tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.00406165, -0.00025477])>]